Segmenting and Clustering Neighborhoods in Toronto 

    Start by creating a new Notebook for this assignment.

    Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe. For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

    To create the above dataframe:
    The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
    Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
    If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe. .

    Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.


## Part I

In [1]:
# Import required libraries

import pandas as pd
import numpy as np
import requests
print('Libraries Imported')

Libraries Imported


In [2]:
# Import Wikipedia Page

get_wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(get_wiki)

In [3]:
# Create a dataframe

df = pd.read_html(req.content, header=0)[0]

In [4]:
# Clean Data set

# Remove boroughs listed as 'Not assigned'
df = df[df.Borough != 'Not assigned']

# Remove neighborhoods listed as 'Not assigned'
df.Neighborhood.replace('Not assigned', df.Borough, inplace=True)

# Group neighborhoods by 'Postal Code'
df=df.groupby("Postal Code").agg(lambda x:','.join(set(x)))


In [5]:
df.head()

,Borough,Neighborhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [6]:
df.shape

(103, 2)

# Part II

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.
In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.
The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example.
Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data
Use the Geocoder package or the csv file to create a dataframe
Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.
Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)



In [7]:
# Import latitude / longtitude data
lat_lon=pd.read_csv("Geospatial_Coordinates.csv")
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
# Add latitude and logitude values to original data frame
df['Latitude']=lat_lon['Latitude'].values
df['Longitude']=lat_lon['Longitude'].values

df.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part III

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

    to add enough Markdown cells to explain what you decided to do and to report any observations you make.
    to generate maps to visualize your neighborhoods and how they cluster together. 

In [18]:
### Import libraries for clustering and mapping
from sklearn.cluster import KMeans
print('completed')

completed


In [19]:
import folium
print('completed')

completed


In [20]:
from geopy.geocoders import Nominatim
print('completed')

completed


In [24]:
# Limit data to just toronto
toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
neighborhoods = toronto_data

address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude
print(lat, lon)

43.6534817 -79.3839347


In [26]:
# Create a map
map_toronto = folium.Map(location=[lat, lon], zoom_start=11)

# Add neighborhood markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [27]:
# Define foursquare credentials
CLIENT_ID = 'S2Z5NXAHC0YLN0TSNRMZTIGYIR4YKTT1OJXLCGAGELHSFG3Z' # your Foursquare ID
CLIENT_SECRET = 'NOGR1PIVL21F44DUABGZJCLDHTCC0XTGMNGX52UHK3GVYYSL' # your Foursquare Secret
VERSION = '20200710' # Foursquare API version

In [28]:
# Explore first neighborhood
toronto_data.loc[0, 'Neighborhood']

'The Beaches'

In [30]:
# Get that neighborhoods lat and lon
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [31]:
### Get top 100 venues from within 500 m of here
LIMIT = 100
radius = 1000

### Create the request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=S2Z5NXAHC0YLN0TSNRMZTIGYIR4YKTT1OJXLCGAGELHSFG3Z&client_secret=NOGR1PIVL21F44DUABGZJCLDHTCC0XTGMNGX52UHK3GVYYSL&v=20200710&ll=43.67635739999999,-79.2930312&radius=1000&limit=100'

In [32]:
# Import JSON library
from pandas.io.json import json_normalize
print('completed')

completed


In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f08e8ddcd5ad5087359bbdd'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 76,
  'suggestedBounds': {'ne': {'lat': 43.685357409000005,
    'lng': -79.28061062898105},
   'sw': {'lat': 43.66735739099998, 'lng': -79.30545177101895}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'lab

In [34]:
# Pull in the categories of our top 100 venues
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
# Clean json and structure it as a new data frame
venues = results['response']['groups'][0]['items']
    
top100_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
top100_venues =top100_venues.loc[:, filtered_columns]

# filter the category for each row
top100_venues['venue.categories'] = top100_venues.apply(get_category_type, axis=1)

# clean columns
top100_venues.columns = [col.split(".")[-1] for col in top100_venues.columns]

top100_venues.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
2,The Beech Tree,Gastropub,43.680493,-79.288846
3,Beaches Bake Shop,Bakery,43.680363,-79.289692
4,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993


In [36]:
# Count venues to confirm
print('{} venues were returned by Foursquare.'.format(top100_venues.shape[0]))

76 venues were returned by Foursquare.


In [37]:
# Create function to repeat for all neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
# For all neighborhoods
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [39]:
# Look at new data frame
print(toronto_venues.shape)
toronto_venues.head()

(3181, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
3,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
4,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop


In [40]:
# Count venues in each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",47,47,47,47,47,47
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


In [41]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


# Explore Neighbourhoods

In [42]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# Group by neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,Berczy Park,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,...,0.01,0.00,0.00,0.010000,0.00,0.00,0.000000,0.00,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.01,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.00,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.00,0.00,0.000000,0.021277,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.00,0.066667,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000
4,Central Bay Street,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.00,0.01,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.020000
5,Christie,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.00,0.020000,0.00,0.00,0.010000,0.00,0.010000,0.000000
6,Church and Wellesley,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.00,0.000000,0.00,0.01,0.000000,0.00,0.000000,0.020000
7,"Commerce Court, Victoria Hotel",0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,...,0.01,0.00,0.00,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.000000
8,Davisville,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.010000,0.01,0.00,0.010000,0.00,0.010000,0.010000
9,Davisville North,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.010000,0.00,0.00,0.010000,0.00,0.010000,0.020000


In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Café,Japanese Restaurant,Hotel,Park,Beer Bar,Gastropub,Bakery,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Restaurant,Bar,Bakery,Furniture / Home Store,Tibetan Restaurant,Gift Shop,Italian Restaurant,Sandwich Place
2,"Business reply mail Processing Centre, South C...",Park,Brewery,Pizza Place,Coffee Shop,Burrito Place,Sushi Restaurant,Fast Food Restaurant,Italian Restaurant,Pub,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Harbor / Marina,Café,Coffee Shop,Garden,Park,Scenic Lookout,Dog Run,Sculpture Garden,Sushi Restaurant,Dance Studio
4,Central Bay Street,Coffee Shop,Park,Clothing Store,Ramen Restaurant,Café,Sushi Restaurant,Gastropub,Pizza Place,Plaza,Cosmetics Shop


In [49]:
# Create function for most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Cluster Neighbourhoods

In [50]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 3, 4, 0, 0, 3, 0, 2, 3, 0])

In [51]:
# Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Coffee Shop,Pizza Place,Beach,Japanese Restaurant,Breakfast Spot,Park,Grocery Store,Burger Joint,Tea Room
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Café,Pub,Bank,Italian Restaurant,Fast Food Restaurant,Ramen Restaurant,Pizza Place,Bakery
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,3,Indian Restaurant,Coffee Shop,Beach,Café,Grocery Store,Burrito Place,Bakery,Brewery,Park,Butcher
3,East Toronto,Studio District,43.659526,-79.340923,3,Coffee Shop,Bar,Café,Brewery,American Restaurant,Diner,Vietnamese Restaurant,Bakery,Italian Restaurant,French Restaurant
4,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Trail,Café,Bookstore,Gym / Fitness Center,College Quad,College Gym,Coffee Shop,Park,Eastern European Restaurant,Doner Restaurant


# Examine Clusters

In [65]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Pub,Coffee Shop,Pizza Place,Beach,Japanese Restaurant,Breakfast Spot,Park,Grocery Store,Burger Joint,Tea Room
1,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Café,Pub,Bank,Italian Restaurant,Fast Food Restaurant,Ramen Restaurant,Pizza Place,Bakery
5,Davisville North,Coffee Shop,Italian Restaurant,Café,Restaurant,Pizza Place,Fast Food Restaurant,Gym,Sushi Restaurant,Dessert Shop,Pub
6,"North Toronto West, Lawrence Park",Coffee Shop,Italian Restaurant,Diner,Mexican Restaurant,Skating Rink,Café,Restaurant,Park,Sporting Goods Shop,Fast Food Restaurant
12,Church and Wellesley,Coffee Shop,Japanese Restaurant,Park,Diner,Yoga Studio,Clothing Store,Sushi Restaurant,Ice Cream Shop,Bookstore,Burger Joint
13,"Regent Park, Harbourfront",Coffee Shop,Diner,Theater,Café,Park,Breakfast Spot,Bakery,Italian Restaurant,Pub,Restaurant
14,"Garden District, Ryerson",Coffee Shop,Japanese Restaurant,Gastropub,Italian Restaurant,Cosmetics Shop,Pizza Place,Poke Place,Plaza,Department Store,Clothing Store
17,Central Bay Street,Coffee Shop,Park,Clothing Store,Ramen Restaurant,Café,Sushi Restaurant,Gastropub,Pizza Place,Plaza,Cosmetics Shop
27,"CN Tower, King and Spadina, Railway Lands, Har...",Harbor / Marina,Café,Coffee Shop,Garden,Park,Scenic Lookout,Dog Run,Sculpture Garden,Sushi Restaurant,Dance Studio
37,"Queen's Park, Ontario Provincial Government",Coffee Shop,Park,Pizza Place,Italian Restaurant,Café,Sushi Restaurant,Ramen Restaurant,Ice Cream Shop,Bookstore,Bubble Tea Shop


In [66]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,Trail,Café,Bookstore,Gym / Fitness Center,College Quad,College Gym,Coffee Shop,Park,Eastern European Restaurant,Doner Restaurant


In [67]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"St. James Town, Cabbagetown",Coffee Shop,Gastropub,Japanese Restaurant,Park,Café,Deli / Bodega,Pub,Restaurant,Diner,Caribbean Restaurant
15,St. James Town,Café,Coffee Shop,Restaurant,Japanese Restaurant,Seafood Restaurant,Cosmetics Shop,Hotel,Italian Restaurant,Theater,Bookstore
16,Berczy Park,Coffee Shop,Restaurant,Café,Japanese Restaurant,Hotel,Park,Beer Bar,Gastropub,Bakery,Farmers Market
18,"Richmond, Adelaide, King",Coffee Shop,Café,Hotel,Theater,Beer Bar,Concert Hall,Clothing Store,Italian Restaurant,Japanese Restaurant,Sushi Restaurant
19,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Café,Hotel,Gym,Park,Theater,Japanese Restaurant,Brewery,Scenic Lookout,Plaza
20,"Toronto Dominion Centre, Design Exchange",Café,Hotel,Coffee Shop,Japanese Restaurant,Restaurant,Concert Hall,Theater,Seafood Restaurant,Gym,Park
21,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,Japanese Restaurant,Concert Hall,Cosmetics Shop,Theater,Plaza
28,Stn A PO Boxes,Coffee Shop,Café,Japanese Restaurant,Hotel,Restaurant,Beer Bar,Seafood Restaurant,Bakery,Farmers Market,Park
29,"First Canadian Place, Underground city",Coffee Shop,Hotel,Café,Japanese Restaurant,Restaurant,Theater,Concert Hall,Monument / Landmark,Gym,Plaza
